In [1]:
pip install openpyxl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 249 kB 53.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.5 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 755 kB 44.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 查看数据
import pandas as pd
dataset = pd.read_excel('中小学文言文语料.xlsx')
print(dataset)

         标题                                                 内容
0      岳阳楼记  庆历四年春，滕子京谪守巴陵郡。越明年，政通人和，百废具兴，乃重修岳阳楼，增其旧制，刻唐贤今人...
1      醉翁亭记  环滁皆山也。其西南诸峰，林壑尤美，望之蔚然而深秀者，琅琊也。山行六七里，渐闻水声潺潺，而泻出...
2     湖心亭看雪  崇祯五年十二月，余住西湖。大雪三日，湖中人鸟声俱绝。是日更定矣，余拏一小舟，拥毳衣炉火，独往...
3     智取生辰纲  次日早，起五更，在府里把担仗都摆在厅前。老都管和两个虞候又将一小担财帛，共十一担，拣了十一个...
4      范进中举  范进进学回家，母亲、妻子俱各欢喜。正待烧锅做饭，只见他丈人胡屠户，手里拿着一副大肠和一瓶酒，...
..      ...                                                ...
97       马说  世有伯乐，然后有千里马。千里马常有，而伯乐不常有。故虽有名马，辱于奴隶之手，骈死于槽枥之间，...
98   送东阳马生序  余幼时即嗜学。家贫，无从致书以观，每假借于藏书之家，手自笔录，计日以还。天大寒，砚冰坚，手指...
99     小石潭记  从小丘西行百二十步，隔篁竹，闻水声，如鸣珮环，心乐之。伐竹取道，下见小潭，水尤清冽。全石以为...
100    岳阳楼记  庆历四年春，滕子京谪守巴陵郡。越明年，政通人和，百废具兴，乃重修岳阳楼，增其旧制，刻唐贤今人...
101    醉翁亭记  环滁皆山也。其西南诸峰，林壑尤美。望之蔚然而深秀者，琅琊也。山行六七里，渐闻水声潺潺，而泄出...

[102 rows x 2 columns]


In [26]:
# 数据处理
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
import random
import torch.utils.data as Data

BATCH_SIZE = 15
LR = 1e-4
NUM_HIDDENS = 256
SEQ_LEN = 40
WORD_DIM = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './'
chinese_data_path = os.path.join(path , '语料.txt')
origin_data , predict_word = [],[]

# vocab
vocab = []
# str to index
s2i = dict()
# index to str
i2s = dict()
index = 0

def get_vocab(path):
    chinese_data_path = path
    with open(chinese_data_path , 'r') as f:
        # print('loading txt...')
        for line in f:
            if len(line) >= 41:
                # 对？进行特殊处理
                line = line.replace("?", "。")
                line = line.replace("？", "。")
                predict_word.append(line[20])
                d = line[:20]+'?'+line[21:40]
                origin_data.append(d)
                for word in line:
                    if word not in vocab:
                        vocab.append(word)
    f.close()
get_vocab(chinese_data_path)

# vocab size
vocab.append('?')
vocab_size = len(vocab)

random.shuffle(vocab)
s2i = {word: i for i, word in enumerate(vocab)}
i2s = {i: word for i, word in enumerate(vocab)}

In [27]:
# 划分数据集
class Dataset(Data.Dataset):
    def __init__(self, data):
        super(Dataset, self).__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x,y = self.data[index]
        # print(x,y)
        x = [s2i[i] for i in x]
        y = s2i[y]
        return torch.LongTensor(x), torch.tensor(y)

data = list(zip(origin_data,predict_word))
random.shuffle(data)
train_dataset = Dataset(data[:100])
test_dataset = Dataset(data[100:])
train_loader = Data.DataLoader(
    dataset=train_dataset,
    shuffle=True,
    num_workers=2,
    batch_size=BATCH_SIZE,
    drop_last=True
)
test_loader = Data.DataLoader(
    dataset=test_dataset,
    shuffle=False,
    num_workers=2,
    batch_size=BATCH_SIZE,
    drop_last=True
)

len(train_dataset), len(test_dataset)

(100, 26)

In [28]:
# 定义函数
import nltk
EPOCH = 10
import torch.nn.functional as F
class BiLSTM(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=WORD_DIM,
            hidden_size=NUM_HIDDENS,
            bidirectional=True
          )
        self.out = nn.Linear(NUM_HIDDENS*2, vocab_size)
    def forward(self,x):
        # print(x.shape)
        # x.shape [batch,seq_len,word_dim]
        # state.shape [1,seq_len,word_dim]
        h_s = torch.randn((1*2,BATCH_SIZE, NUM_HIDDENS),device=device)
        c_s = torch.randn((1*2,BATCH_SIZE, NUM_HIDDENS),device=device)
        output,(_,_)= self.lstm(x.transpose(1,0),(h_s,c_s)) # output.shape [seq_len,batch,num_hiddens]
        chinese = self.out(output[20])  # chinese.shape [batch,vocab_size]

        return chinese

def encode(data,embed):
    return embed(data)

def train_porcess(pred,acc,device):
    # pred.shape [batch,vocab_size]
    # acc.shape [batch]
    # print(pred.shape)
    pred = F.softmax(pred, dim=-1)
    # print(pred.shape)
    pred = pred.argmax(dim=-1).cpu().numpy()
    acc = acc.cpu().numpy()
    hypothesis = [i2s[i] for i in pred.tolist()]
    reference = [i2s[i] for i in acc.tolist()]
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
    print('预测下一个字:',[i2s[i] for i in pred.tolist()])
    print('实际下一个字:',[i2s[i] for i in acc.tolist()])
    print('BLEU值为：',BLEUscore)

In [29]:
# 模型实例化
bilstm = BiLSTM(vocab_size)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(bilstm.parameters(), lr=LR)
embed = nn.Embedding(vocab_size, WORD_DIM, device = device)
# 保存整个模型 torch.save(bilstm, "BiLSTM.pth")
torch.save(bilstm.state_dict(),"BiLSTM.pth")
output_model = './BiLSTM.pth'

In [30]:
# 训练模型
BATCH_SIZE = 15
if __name__ == '__main__':

    bestscore = 0
    # output_model = './BiLSTM.pth'
    state_dict = torch.load(output_model, map_location='cpu')
    # print(checkpoint)
    bilstm.load_state_dict(torch.load("BiLSTM.pth"))
    bilstm.to(device)
    bilstm.train()
    for epoch in range(10):
        print(f'epoch = {epoch}')
        accuracy = 0
        time = 0
        for step, datas in enumerate(train_loader):
            data, label = tuple(t.to(device)for t in datas)
            data = encode(data, embed)
            chinese = bilstm(data)
            loss = criterion(chinese, label.long())
            optim.zero_grad()
            loss.backward()
            optim.step()
            if step%10 == 0:
                print(f'loss = {loss}')

            if step%50 == 0:
                train_porcess(chinese, label, device)
        bilstm.eval()
        for step, datas in enumerate(test_loader):
            # print(datas)
            data, label = tuple(t.to(device)for t in datas)
            data = encode(data, embed)
            chinese = bilstm(data)

            pred = F.softmax(chinese, dim=-1)
            pred = pred.argmax(dim=-1).cpu().numpy()
            acc = label.cpu().numpy()
            accuracy += sum(acc==pred)
            time += len(acc)
            accuracy/=time+0.0
        # 百分数形式 print('准确度为：{:.2f}%\n'.format(accuracy * 100))
        print('准确度为：',accuracy)
        print('\n')
        if accuracy>bestscore:
            torch.save(
            {
                'model_state_dict': bilstm.state_dict(),
                'optimizer_state_dict': optim.state_dict()
            }, output_model
          )
            bestscore = accuracy
        bilstm.train()

epoch = 0
loss = 7.768373012542725
预测下一个字: ['饵', '倾', '饵', '绮', '饵', '饵', '饵', '耒', '饵', '饵', '孩', '耒', '耒', '叩', '饵']
实际下一个字: ['节', '。', '窥', '日', '檐', '其', '者', '叠', '之', '山', '怒', '学', '孔', '吹', '嬉']
BLEU值为： 0
准确度为： 0.0


epoch = 1
loss = 7.780065059661865
预测下一个字: ['耒', '雅', '饵', '叩', '饵', '楚', '饵', '耒', '饵', '叩', '饵', '饵', '饵', '饵', '绮']
实际下一个字: ['以', '狙', '归', '飞', '应', '与', '，', '孔', '气', '也', '通', '折', '》', '少', '横']
BLEU值为： 0
准确度为： 0.0


epoch = 2
loss = 7.6887736320495605
预测下一个字: ['攥', '饵', '倾', '饵', '饵', '饵', '/', '山', '辞', '旧', '气', '怒', '楚', '耒', '在']
实际下一个字: ['其', '大', '斯', '檐', '山', '其', '者', '公', '伯', '尝', '。', '。', '与', '理', '顷']
BLEU值为： 9.257324954728539e-232
准确度为： 0.06666666666666667


epoch = 3
loss = 7.614497184753418
预测下一个字: ['叩', '辞', '怒', '，', '叠', '怒', '，', '辞', '耒', '，', '怒', '辞', '怒', '借', '嬉']
实际下一个字: ['遍', '菊', '名', '。', '哉', '至', '尝', '借', '学', '。', '更', '政', '怒', '数', '节']
BLEU值为： 1.1008876702055895e-231
准确度为： 0.06666666666666667


epoch = 4
loss = 7.57454